In [1]:
import json
with open("./shortanswer_ratings_cache.json", 'r', encoding='utf-8') as f:
    sa_cache = json.load(f)

sa_qdict = {}
for qaid, qdata in sa_cache.items():
    qid = qdata['qid']
    if qid not in sa_qdict:
        sa_qdict[qid] = {"question": qdata['question'], "correct_answer": qdata['correct_answer']}

In [4]:
with open("./compiled_results_sqa/grok-3-latest_phase1_compiled.json", 'r', encoding='utf-8') as f:
    claude_results = json.load(f)

for qid in claude_results["results"].keys():
    if qid not in sa_qdict:
        print(f"Warning: {qid} not found in sa_qdict")


In [ ]:
import re
from base_game_class import BaseGameClass

judge_model = "claude-opus-4-20250514"
sa_to_mc_file = "./SimpleMC.jsonl"

fout = open(sa_to_mc_file, 'a', encoding='utf-8')

judge = BaseGameClass(subject_id=None, subject_name=judge_model, is_human_player=False, log_dir=None)
sysprompt=""
prompt = """I need your help turning a short-answer quiz into a multiple-choice quiz. I'm going to show you a question and its correct answer, and I want you to generate three distractors. 
Each distractor should be a plausible answer that is NOT the correct answer. Each should be the same \"type\" of answer as the correct answer (e.g., date, person name, number, etc), and follow the format of the correct answer.
Output each distractor a separate line, and do not include any other text. Your entire response should be just the distractors, one per line.
Here is the question and correct answer:

Question: {question}

Correct Answer: {correct_answer}

Distractors:
"""
sqa_test_2427_check = False
for ctr, (qid, qdata) in enumerate(sa_qdict.items()):
    if qid == "sqa_test_2427": sqa_test_2427_check = True
    if not sqa_test_2427_check: continue
    question = qdata['question']
    correct_answer = qdata['correct_answer'].strip()
    print(f"Processing question {ctr+1}, ID: {qid}")
    
    while True:
        resp, _, _ = judge._get_llm_answer(options=None, q_text=prompt.format(question=question, correct_answer=correct_answer), message_history=[], keep_appending=False, setup_text=sysprompt, MAX_TOKENS=None, temp=1.0)
        ans_list = re.split(r'\n\s*\n+', resp.strip()) if resp else []
        ans_set = set([a.strip().upper() for a in ans_list] + [correct_answer.upper()]) 
        if len(ans_set) == 4 and all(ans.strip() for ans in ans_list):
            break
        else:
            print(f"Invalid response format for question {qid}. Retrying...")
    fout.write(json.dumps({"qid": qid, "question": question, "correct_answer": correct_answer, "distractors": ans_list}, ensure_ascii=False) + "\n")
    fout.flush()


Provider: Anthropic
Processing question 4, ID: sqa_test_2660
In model_call, provider=Anthropic, attempt=1
Processing question 5, ID: sqa_test_2237
In model_call, provider=Anthropic, attempt=1
Processing question 6, ID: sqa_test_3445
In model_call, provider=Anthropic, attempt=1
Processing question 7, ID: sqa_test_2894
In model_call, provider=Anthropic, attempt=1
Processing question 8, ID: sqa_test_3128
In model_call, provider=Anthropic, attempt=1
Processing question 9, ID: sqa_test_4168
In model_call, provider=Anthropic, attempt=1
Processing question 10, ID: sqa_test_2791
In model_call, provider=Anthropic, attempt=1
Processing question 11, ID: sqa_test_2097
In model_call, provider=Anthropic, attempt=1
Processing question 12, ID: sqa_test_2383
In model_call, provider=Anthropic, attempt=1
Processing question 13, ID: sqa_test_3406
In model_call, provider=Anthropic, attempt=1
Processing question 14, ID: sqa_test_3494
In model_call, provider=Anthropic, attempt=1
Processing question 15, ID: s

KeyboardInterrupt: 

In [7]:
import re
re.split(r'\n\s*\n+', resp.strip())

['"Power is everything, Dante. Without it, you are nothing. The weak deserve only death, and you have proven your weakness today."',
 '"You disappoint me, brother. Father\'s blood runs thin in your veins. Only through power can one achieve true greatness."',
 '"This is the difference between us, Dante. I embrace my demon heritage while you cling to your pathetic humanity. You will never defeat me."']